Ürün ratinglerini daha doğru hesaplamaya 
çalışmak ve ürün yorumlarını daha doğru 
sıralamak.

In [1]:
import pandas as pd
import math
import scipy.stats as st

In [ ]:
df = pd.read_csv("amazon_review.csv")

In [3]:
df.shape

(4915, 12)

In [8]:
df.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0
5,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,"[0, 0]",It's mini storage. It doesn't do anything els...,5.0,Not a lot to really be said,1367193600,2013-04-29,588,0,0
6,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,"[0, 0]",I have it in my phone and it never skips a bea...,5.0,Works well,1382140800,2013-10-19,415,0,0
7,A3CW0ZLUO5X2B1,B007WTAJTO,"35-year Technology Consumer ""8-tracks to 802.11""","[0, 0]",It's hard to believe how affordable digital ha...,5.0,32 GB for less than two sawbucks...what's not ...,1404950400,2014-10-07,62,0,0
8,A2CYJO155QP33S,B007WTAJTO,4evryoung,"[1, 1]",Works in a HTC Rezound. Was running short of ...,5.0,Loads of room,1395619200,2014-03-24,259,1,1
9,A2S7XG3ZC4VGOQ,B007WTAJTO,53rdcard,"[0, 0]","in my galaxy s4, super fast card, and am total...",5.0,works great,1381449600,2013-11-10,393,0,0


In [9]:
df["overall"].mean()

4.587589013224822

In [13]:
df["overall"].value_counts()

5.0    3922
4.0     527
1.0     244
3.0     142
2.0      80
Name: overall, dtype: int64

In [19]:
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

In [22]:
df["reviewTime"].max()

Timestamp('2014-12-07 00:00:00')

In [23]:
df["reviewTime"].min()

Timestamp('2012-01-09 00:00:00')

In [30]:
today_date = pd.to_datetime('2015-01-10 0:0:0')

Veri setindeki yorumlar 2012 - 2014 yılları arasında yapılmıştır. Analiz yapılacak tarihi 2016.01.10 olarak belirledik.

In [31]:
df["days"] = (today_date - df['reviewTime']).dt.days

In [41]:
df["days"].mean()

470.3670396744659

In [32]:
df["days"].min()

34

In [33]:
df["days"].max()

1097

Veri setindeki ürüne gelen yorumları tarihlerin güncelliğine göre ağırlıklandırarak daha sağlıklı ortalama hesabı yapılabilir. Bunun için "days" değişkenine ait çeyreklik değerler hesaplanır. 

In [57]:
df["days"].quantile([0.25, 0.50, 0.75])

0.25    314.0
0.50    464.0
0.75    634.0
Name: days, dtype: float64

In [58]:
def time_based_weighted_average(dataframe, w1=30, w2=26, w3=24, w4=20):
    return dataframe.loc[df["days"] <= 314, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["days"] > 314) & (dataframe["days"] <= 464), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["days"] > 464) & (dataframe["days"] <= 634), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["days"] > 634), "overall"].mean() * w4 / 100

In [59]:
time_based_weighted_average(df, w1=30, w2=26, w3=24, w4=20)

4.600583941300071

İlk ortalama değeri yeni hesapladığımız ortalama değerden düşüktür. Yorumlardaki zamanın etkisini de hesaba katarak daha sağlıklı bir hesap yapılmıştır.

In [65]:
df.sort_values("helpful_yes",ascending= False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,735
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,612
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,836
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1066
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,810
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,840
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0,I own it,1393545600,2014-02-28,283,60,68,316
2909,A10B6G6W3DW1EY,B007WTAJTO,Luopo,"[53, 236]",I know armed with this in my Android tablet an...,4.0,Win Win situation,1393200000,2014-02-24,287,53,236,320
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0,Awesome Card!,1339200000,2012-09-06,823,51,65,856
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,191


In [94]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up+down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [95]:
comments = pd.DataFrame({"up": df["helpful_yes"], "down": df["total_vote"]-df["helpful_yes"]})

In [96]:
comments.sort_values("up" ,ascending= False).head(20)

,up,down
2031,1952,68
4212,1568,126
3449,1428,77
317,422,73
3981,112,27
4596,82,27
1835,60,8
2909,53,183
4306,51,14
4672,45,4


In [97]:
comments["wilson_lower_bound"] = comments.apply(lambda x: wilson_lower_bound(x["up"], x["down"]), axis=1)

In [98]:
comments.sort_values("wilson_lower_bound",ascending= False).head(20)

,up,down,wilson_lower_bound
2031,1952,68,0.957544
3449,1428,77,0.936519
4212,1568,126,0.912139
317,422,73,0.818577
4672,45,4,0.808109
1835,60,8,0.784651
3981,112,27,0.732136
3807,22,3,0.700442
4306,51,14,0.670334
4596,82,27,0.663595
